In [2]:

import json
import pandas as pd
import numpy as np

def load_jsonl(file_path: str) -> list[list[dict]]:
    """
    Load and parse a file where each line is a JSON-encoded string representing
    a participant's response data across trials.
    """
    participants_data = []
    with open(file_path, "r") as file:
        for line in file:
            try:
                participant_data = json.loads(line.strip())
                participants_data.append(participant_data)
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {e}")
    return participants_data


def retrieve_conditions(participants_data: list[list[dict]]) -> list[str]:
    """
    Retrieve the condition of each participant from the data.
    """
    conditions = []
    for participant_data in participants_data:
        condition = next(
            (
                ["Casual", "Competitive"][entry["condition"]]
                for entry in participant_data
                if entry.get("condition", None) is not None
            ),
            None,
        )
        assert condition is not None, "Condition not found for participant"
        conditions.append(condition)
    return conditions


def retrieve_subj_demographics(participants_data: list[list[dict]]) -> list[tuple]:
    """
    Returns the self-reported demographics of each participant.
    """
    full_demographics = []
    for participant_data in participants_data:
        demographics = [
            (
                entry["response"]["race"],
                entry["response"]["gender"],
                int(entry["response"]["age"]),
                entry["response"]["college"],
            )
            for entry in participant_data
            if type(entry.get("response")) is dict
            and entry["response"].get("race") is not None
        ]
        full_demographics.append(demographics[0])
    return full_demographics


def extract_race_gender_age(url):
    """
    Extract the race, gender, and age from a character's image URL.
    """
    filename = url.split("/")[-1].split(".")[0].lower()
    return filename.split("_")[2:]


def compare_race(raceA: str, raceB: str) -> bool:
    """
    Compare two races to check for a match.
    """
    codes = {
        "South Asian": 0,
        "south-asian": 0,
        "East/Southeast Asian": 1,
        "east-asian": 1,
        "Black": 2,
        "black": 2,
        "White": 3,
        "white": 3,
        "Hispanic/Latine/Latinx": 4,
        "latino": 4,
        "Indigenous": 5,
        "Multiracial": 6,
        "Other": 7,
    }
    return codes.get(raceA, np.nan) == codes.get(raceB, np.nan)


def convert_age(age_int: int) -> str:
    """
    Convert age integer into age range.
    """
    if age_int <= 24:
        return "18-24"
    elif age_int <= 31:
        return "25-31"
    elif age_int <= 38:
        return "32-38"
    elif age_int <= 45:
        return "39-45"
    return "45+"


def convert_gender(gender: str) -> str:
    """
    Convert gender into standardized categories.
    """
    genders = ["Man", "Woman", "Non-binary", "Other"]
    codes = {
        "Man": 0,
        "male": 0,
        "man": 0,
        "Woman": 1,
        "woman": 1,
        "female": 1,
        "Non-binary": 2,
        "Other": 3,
    }
    return genders[codes.get(gender)]


# Path to the dataset
jatos_data_path = "Team_Building_16_11_2024_a.jsonl"

# Load the data
data = load_jsonl(jatos_data_path)
conditions = retrieve_conditions(data)
subject_demographics = retrieve_subj_demographics(data)

# Dictionary to store extracted information for each decision
merged = {
    "subject": [],
    "condition": [],
    "subject_race": [],
    "subject_age": [],
    "subject_gender": [],
    "chosen_race": [],
    "chosen_age": [],
    "chosen_gender": [],
    "rejected_race": [],
    "rejected_age": [],
    "rejected_gender": [],
    "shared_features_chosen": [],
    "shared_features_rejected": [],
}

# Extract decision-level data
for i, participant_data in enumerate(data):
    subject_id = i + 1
    condition = conditions[i]
    subject_race, subject_gender, subject_age, _ = subject_demographics[i]

    for entry in participant_data:
        if entry.get("winner") is not None:
            # Extract chosen and rejected characters
            winner_url = entry['left_image'] if entry['choice'] == 'left' else entry['right_image']
            loser_url = entry['left_image'] if entry['choice'] == 'right' else entry['right_image']

            winner_demo = extract_race_gender_age(winner_url)
            loser_demo = extract_race_gender_age(loser_url)

            if not winner_demo or not loser_demo:
                continue

            # Unpack winner and loser demographics
            chosen_race, chosen_gender, chosen_age = winner_demo
            rejected_race, rejected_gender, rejected_age = loser_demo

            # Calculate shared features
            shared_chosen = 0
            shared_chosen += compare_race(subject_race, chosen_race)
            shared_chosen += convert_age(subject_age) == convert_age(int(chosen_age))
            shared_chosen += convert_gender(subject_gender) == convert_gender(chosen_gender)

            shared_rejected = 0
            shared_rejected += compare_race(subject_race, rejected_race)
            shared_rejected += convert_age(subject_age) == convert_age(int(rejected_age))
            shared_rejected += convert_gender(subject_gender) == convert_gender(rejected_gender)

            # Append to merged data
            merged["subject"].append(subject_id)
            merged["condition"].append(condition)
            merged["subject_race"].append(subject_race)
            merged["subject_age"].append(convert_age(subject_age))
            merged["subject_gender"].append(convert_gender(subject_gender))
            merged["chosen_race"].append(chosen_race)
            merged["chosen_age"].append(convert_age(int(chosen_age)))
            merged["chosen_gender"].append(convert_gender(chosen_gender))
            merged["rejected_race"].append(rejected_race)
            merged["rejected_age"].append(convert_age(int(rejected_age)))
            merged["rejected_gender"].append(convert_gender(rejected_gender))
            merged["shared_features_chosen"].append(shared_chosen)
            merged["shared_features_rejected"].append(shared_rejected)

# Convert the dictionary into a DataFrame
full_data = pd.DataFrame(merged)

# Display the first few rows
full_data.head()


,subject,condition,subject_race,subject_age,subject_gender,chosen_race,chosen_age,chosen_gender,rejected_race,rejected_age,rejected_gender,shared_features_chosen,shared_features_rejected
0,1,Casual,White,32-38,Man,black,39-45,Man,south-asian,45+,Woman,1,0
1,1,Casual,White,32-38,Man,white,39-45,Man,east-asian,45+,Woman,2,0
2,1,Casual,White,32-38,Man,latino,18-24,Man,black,32-38,Woman,1,1
3,1,Casual,White,32-38,Man,east-asian,45+,Woman,black,39-45,Man,0,1
4,1,Casual,White,32-38,Man,black,18-24,Woman,black,32-38,Woman,0,1


In [21]:
data.to_csv('experiments/sortablerank/fifth_pass_data.csv', index=False)